# 3 Body Problem

Lagrange Triangular Solution


In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using OrdinaryDiffEq
using ModelingToolkit: D_nounits as D, t_nounits as t
# ─── 1. Define the 3‑Body ODE System ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Softened distances to prevent numerical explosions
dx12, dy12 = x2 - x1, y2 - y1
dx13, dy13 = x3 - x1, y3 - y1
dx23, dy23 = x3 - x2, y3 - y2

r12_sq = dx12^2 + dy12^2 
r13_sq = dx13^2 + dy13^2 
r23_sq = dx23^2 + dy23^2 

r12 = r12_sq^(3/2)
r13 = r13_sq^(3/2)
r23 = r23_sq^(3/2)

eqs = [
    D(x1) ~ vx1,  D(y1) ~ vy1,
    D(x2) ~ vx2,  D(y2) ~ vy2,
    D(x3) ~ vx3,  D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12 + m3*dx13/r13),
    D(vy1) ~ G*(m2*dy12/r12 + m3*dy13/r13),

    D(vx2) ~ G*(m3*dx23/r23 - m1*dx12/r12),
    D(vy2) ~ G*(m3*dy23/r23 - m1*dy12/r12),

    D(vx3) ~ G*(-m1*dx13/r13 - m2*dx23/r23),
    D(vy3) ~ G*(-m1*dy13/r13 - m2*dy23/r23),
]

@mtkbuild threebody = ODESystem(eqs, t)



# Initial conditions that lead to chaotic motion
# Modified parameters and initial conditions
Gval =1
masses = [m1 => 1.0, m2 => 1.0, m3 => 1.0]
θ = 2π/3  # 120° separation
ω = 1.0   # Angular velocity

u0 = [
    x1 => cos(0),        y1 => sin(0),        vx1 => -ω*sin(0),        vy1 => ω*cos(0),
    x2 => cos(θ),        y2 => sin(θ),        vx2 => -ω*sin(θ),        vy2 => ω*cos(θ),
    x3 => cos(2θ),       y3 => sin(2θ),       vx3 => -ω*sin(2θ),       vy3 => ω*cos(2θ)
]

tspan = (0.0, 70.0)
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, ImplicitMidpoint(), dt=0.05) 

# ─── 3. Animate Chaotic Motion ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Lagrange Triangular Solution",
    aspect=DataAspect(),
    limits=(-10..10, -10..10)
)

# Initial positions
pos1 = Observable(Point2f(sol[x1][1], sol[y1][1]))
pos2 = Observable(Point2f(sol[x2][1], sol[y2][1]))
pos3 = Observable(Point2f(sol[x3][1], sol[y3][1]))

# Trails with limited length
trail_length = 150
trail1 = Observable([Point2f(sol[x1][1], sol[y1][1])])
trail2 = Observable([Point2f(sol[x2][1], sol[y2][1])])
trail3 = Observable([Point2f(sol[x3][1], sol[y3][1])])

colors = [(:orange, :yellow), (:blue, :cyan), (:red, :pink)]

for (i, (pos, trail, (c1, c2))) in enumerate(zip([pos1, pos2, pos3], 
                                                [trail1, trail2, trail3], 
                                                colors))
    GLMakie.scatter!(ax, pos, color=c1, markersize=15, strokecolor=:black, strokewidth=1)
    GLMakie.lines!(ax, trail, color=c2, linewidth=1.5)
end

record(fig, "Lagrange Triangular Solution.gif", eachindex(sol.t); framerate=30) do i
    # Update positions
    pos1[] = Point2f(sol[x1][i], sol[y1][i])
    pos2[] = Point2f(sol[x2][i], sol[y2][i])
    pos3[] = Point2f(sol[x3][i], sol[y3][i])

    # Update trails (maintain limited length)
    for trail in [trail1, trail2, trail3]
        if length(trail[]) > trail_length
            trail[] = trail[][end-trail_length+1:end]
        end
    end
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])

    # Notify changes
    notify.([pos1, pos2, pos3])
    notify.([trail1, trail2, trail3])
end

 Figure-8 Orbit (Chenciner-Montgomery Orbit)

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t
# ─── 1. Define the 3‑Body ODE System ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Softened distances to prevent numerical explosions
dx12, dy12 = x2 - x1, y2 - y1
dx13, dy13 = x3 - x1, y3 - y1
dx23, dy23 = x3 - x2, y3 - y2

r12_sq = dx12^2 + dy12^2 
r13_sq = dx13^2 + dy13^2 
r23_sq = dx23^2 + dy23^2 

r12 = r12_sq^(3/2)
r13 = r13_sq^(3/2)
r23 = r23_sq^(3/2)

eqs = [
    D(x1) ~ vx1,  D(y1) ~ vy1,
    D(x2) ~ vx2,  D(y2) ~ vy2,
    D(x3) ~ vx3,  D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12 + m3*dx13/r13),
    D(vy1) ~ G*(m2*dy12/r12 + m3*dy13/r13),

    D(vx2) ~ G*(m3*dx23/r23 - m1*dx12/r12),
    D(vy2) ~ G*(m3*dy23/r23 - m1*dy12/r12),

    D(vx3) ~ G*(-m1*dx13/r13 - m2*dx23/r23),
    D(vy3) ~ G*(-m1*dy13/r13 - m2*dy23/r23),
]

@mtkbuild threebody = ODESystem(eqs, t)



# Initial conditions that lead to chaotic motion
# Modified parameters and initial conditions
Gval = 1
masses = [m1 => 1.0, m2 => 1.0, m3 => 1.0]
u0 = [
    x1 => -0.97000436, y1 => 0.24308753, vx1 => 0.46620368, vy1 => 0.43236573,
    x2 => 0.97000436,  y2 => -0.24308753, vx2 => 0.46620368, vy2 => 0.43236573,
    x3 => 0.0,         y3 => 0.0,         vx3 => -0.93240737, vy3 => -0.86473146
]

tspan = (0.0, 50.0)
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, ImplicitMidpoint(), dt=0.05)  

# ─── 3. Animate Chaotic Motion ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Figure-8 Orbit (Chenciner-Montgomery Orbit)",
    aspect=DataAspect(),
    limits=(-5..5, -5..5)
)

# Initial positions
pos1 = Observable(Point2f(sol[x1][1], sol[y1][1]))
pos2 = Observable(Point2f(sol[x2][1], sol[y2][1]))
pos3 = Observable(Point2f(sol[x3][1], sol[y3][1]))

# Trails with limited length
trail_length = 150
trail1 = Observable([Point2f(sol[x1][1], sol[y1][1])])
trail2 = Observable([Point2f(sol[x2][1], sol[y2][1])])
trail3 = Observable([Point2f(sol[x3][1], sol[y3][1])])

colors = [(:orange, :yellow), (:blue, :cyan), (:red, :pink)]

for (i, (pos, trail, (c1, c2))) in enumerate(zip([pos1, pos2, pos3], 
                                                [trail1, trail2, trail3], 
                                                colors))
    GLMakie.scatter!(ax, pos, color=c1, markersize=15, strokecolor=:black, strokewidth=1)
    GLMakie.lines!(ax, trail, color=c2, linewidth=1.5)
end

record(fig, "Figure-8 Orbit (Chenciner-Montgomery Orbit).gif", eachindex(sol.t); framerate=30) do i
    # Update positions
    pos1[] = Point2f(sol[x1][i], sol[y1][i])
    pos2[] = Point2f(sol[x2][i], sol[y2][i])
    pos3[] = Point2f(sol[x3][i], sol[y3][i])

    # Update trails (maintain limited length)
    for trail in [trail1, trail2, trail3]
        if length(trail[]) > trail_length
            trail[] = trail[][end-trail_length+1:end]
        end
    end
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])

    # Notify changes
    notify.([pos1, pos2, pos3])
    notify.([trail1, trail2, trail3])
end

Pythagorean Triple Orbit

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t
# ─── 1. Define the 3‑Body ODE System ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Softened distances to prevent numerical explosions
dx12, dy12 = x2 - x1, y2 - y1
dx13, dy13 = x3 - x1, y3 - y1
dx23, dy23 = x3 - x2, y3 - y2

r12_sq = dx12^2 + dy12^2 
r13_sq = dx13^2 + dy13^2 
r23_sq = dx23^2 + dy23^2 

r12 = r12_sq^(3/2)
r13 = r13_sq^(3/2)
r23 = r23_sq^(3/2)

eqs = [
    D(x1) ~ vx1,  D(y1) ~ vy1,
    D(x2) ~ vx2,  D(y2) ~ vy2,
    D(x3) ~ vx3,  D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12 + m3*dx13/r13),
    D(vy1) ~ G*(m2*dy12/r12 + m3*dy13/r13),

    D(vx2) ~ G*(m3*dx23/r23 - m1*dx12/r12),
    D(vy2) ~ G*(m3*dy23/r23 - m1*dy12/r12),

    D(vx3) ~ G*(-m1*dx13/r13 - m2*dx23/r23),
    D(vy3) ~ G*(-m1*dy13/r13 - m2*dy23/r23),
]

@mtkbuild threebody = ODESystem(eqs, t)



# Initial conditions that lead to chaotic motion
# Modified parameters and initial conditions
Gval = 1.0
# Mass ratios
masses = [m1 => 3.0, m2 => 4.0, m3 => 5.0]
u0 = [
    x1 => 1.0,  y1 => 3.0,  vx1 => 0.0,  vy1 => 0.0,
    x2 => -2.0, y2 => -1.0, vx2 => 0.0,  vy2 => 0.0,
    x3 => 1.0,  y3 => -1.0, vx3 => 0.0,  vy3 => 0.0
]

tspan = (0.0, 50.0)
p = [G => Gval, m1 => 3.0, m2 => 4.0, m3 => 5.0]

prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, Vern7(), reltol=1e-8, abstol=1e-8, saveat=0.05)

# ─── 3. Animate Chaotic Motion ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Pythagorean Triple Orbit",
    aspect=DataAspect(),
    limits=(-5..5, -5..5)
)

# Initial positions
pos1 = Observable(Point2f(sol[x1][1], sol[y1][1]))
pos2 = Observable(Point2f(sol[x2][1], sol[y2][1]))
pos3 = Observable(Point2f(sol[x3][1], sol[y3][1]))

# Trails with limited length
trail_length = 150
trail1 = Observable([Point2f(sol[x1][1], sol[y1][1])])
trail2 = Observable([Point2f(sol[x2][1], sol[y2][1])])
trail3 = Observable([Point2f(sol[x3][1], sol[y3][1])])

colors = [(:orange, :yellow), (:blue, :cyan), (:red, :pink)]

for (i, (pos, trail, (c1, c2))) in enumerate(zip([pos1, pos2, pos3], 
                                                [trail1, trail2, trail3], 
                                                colors))
    GLMakie.scatter!(ax, pos, color=c1, markersize=15, strokecolor=:black, strokewidth=1)
    GLMakie.lines!(ax, trail, color=c2, linewidth=1.5)
end

record(fig, "Pythagorean Triple Orbit.gif", eachindex(sol.t); framerate=30) do i
    # Update positions
    pos1[] = Point2f(sol[x1][i], sol[y1][i])
    pos2[] = Point2f(sol[x2][i], sol[y2][i])
    pos3[] = Point2f(sol[x3][i], sol[y3][i])

    # Update trails (maintain limited length)
    for trail in [trail1, trail2, trail3]
        if length(trail[]) > trail_length
            trail[] = trail[][end-trail_length+1:end]
        end
    end
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])

    # Notify changes
    notify.([pos1, pos2, pos3])
    notify.([trail1, trail2, trail3])
end

Broucke-Hénon "Bouncing" Orbit

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t
# ─── 1. Define the 3‑Body ODE System ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Softened distances to prevent numerical explosions
dx12, dy12 = x2 - x1, y2 - y1
dx13, dy13 = x3 - x1, y3 - y1
dx23, dy23 = x3 - x2, y3 - y2

r12_sq = dx12^2 + dy12^2 
r13_sq = dx13^2 + dy13^2 
r23_sq = dx23^2 + dy23^2 

r12 = r12_sq^(3/2)
r13 = r13_sq^(3/2)
r23 = r23_sq^(3/2)

eqs = [
    D(x1) ~ vx1,  D(y1) ~ vy1,
    D(x2) ~ vx2,  D(y2) ~ vy2,
    D(x3) ~ vx3,  D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12 + m3*dx13/r13),
    D(vy1) ~ G*(m2*dy12/r12 + m3*dy13/r13),

    D(vx2) ~ G*(m3*dx23/r23 - m1*dx12/r12),
    D(vy2) ~ G*(m3*dy23/r23 - m1*dy12/r12),

    D(vx3) ~ G*(-m1*dx13/r13 - m2*dx23/r23),
    D(vy3) ~ G*(-m1*dy13/r13 - m2*dy23/r23),
]

@mtkbuild threebody = ODESystem(eqs, t)



# Initial conditions that lead to chaotic motion
# Modified parameters and initial conditions
Gval = 1.0
# Mass ratios
masses = [m1 => 1.0, m2 => 1.0, m3 => 1.0]
u0 = [
    x1 => -1.0, y1 => 0.0, vx1 => 0.347111, vy1 => 0.532728,
    x2 => 1.0,  y2 => 0.0, vx2 => 0.347111, vy2 => 0.532728,
    x3 => 0.0,  y3 => 0.0, vx3 => -0.694222, vy3 => -1.065456
]

tspan = (0.0, 50.0)
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, ImplicitMidpoint(), dt=0.05)  


# ─── 3. Animate Chaotic Motion ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Broucke-Hénon Bouncing Orbit",
    aspect=DataAspect(),
    limits=(-5..5, -5..5)
)

# Initial positions
pos1 = Observable(Point2f(sol[x1][1], sol[y1][1]))
pos2 = Observable(Point2f(sol[x2][1], sol[y2][1]))
pos3 = Observable(Point2f(sol[x3][1], sol[y3][1]))

# Trails with limited length
trail_length = 150
trail1 = Observable([Point2f(sol[x1][1], sol[y1][1])])
trail2 = Observable([Point2f(sol[x2][1], sol[y2][1])])
trail3 = Observable([Point2f(sol[x3][1], sol[y3][1])])

colors = [(:orange, :yellow), (:blue, :cyan), (:red, :pink)]

for (i, (pos, trail, (c1, c2))) in enumerate(zip([pos1, pos2, pos3], 
                                                [trail1, trail2, trail3], 
                                                colors))
    GLMakie.scatter!(ax, pos, color=c1, markersize=15, strokecolor=:black, strokewidth=1)
    GLMakie.lines!(ax, trail, color=c2, linewidth=1.5)
end

record(fig, "Broucke-Hénon Bouncing Orbit.gif", eachindex(sol.t); framerate=30) do i
    # Update positions
    pos1[] = Point2f(sol[x1][i], sol[y1][i])
    pos2[] = Point2f(sol[x2][i], sol[y2][i])
    pos3[] = Point2f(sol[x3][i], sol[y3][i])

    # Update trails (maintain limited length)
    for trail in [trail1, trail2, trail3]
        if length(trail[]) > trail_length
            trail[] = trail[][end-trail_length+1:end]
        end
    end
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])

    # Notify changes
    notify.([pos1, pos2, pos3])
    notify.([trail1, trail2, trail3])
end

Lagrange Equilateral Triangle (Rotating Frame)

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t
# ─── 1. Define the 3‑Body ODE System ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Softened distances to prevent numerical explosions
dx12, dy12 = x2 - x1, y2 - y1
dx13, dy13 = x3 - x1, y3 - y1
dx23, dy23 = x3 - x2, y3 - y2

r12_sq = dx12^2 + dy12^2 
r13_sq = dx13^2 + dy13^2 
r23_sq = dx23^2 + dy23^2 

r12 = r12_sq^(3/2)
r13 = r13_sq^(3/2)
r23 = r23_sq^(3/2)

eqs = [
    D(x1) ~ vx1,  D(y1) ~ vy1,
    D(x2) ~ vx2,  D(y2) ~ vy2,
    D(x3) ~ vx3,  D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12 + m3*dx13/r13),
    D(vy1) ~ G*(m2*dy12/r12 + m3*dy13/r13),

    D(vx2) ~ G*(m3*dx23/r23 - m1*dx12/r12),
    D(vy2) ~ G*(m3*dy23/r23 - m1*dy12/r12),

    D(vx3) ~ G*(-m1*dx13/r13 - m2*dx23/r23),
    D(vy3) ~ G*(-m1*dy13/r13 - m2*dy23/r23),
]

@mtkbuild threebody = ODESystem(eqs, t)



# Initial conditions that lead to chaotic motion
# Modified parameters and initial conditions
Gval = 1.0
# Mass ratios
masses = [m1 => 1.0, m2 => 1.0, m3 => 1.0]
# Equilateral triangle configuration
a = 1.0  # Distance from center to vertices
total_mass = 3.0 
ω = sqrt(Gval * total_mass / (2 * sqrt(3) * a^3))
# Initial positions (equilateral triangle)
u0 = [
    # Body 1
    x1 => a * cos(0),         y1 => a * sin(0), 
    vx1 => -a*ω*sin(0),       vy1 => a*ω*cos(0),
    
    # Body 2
    x2 => a * cos(2π/3),      y2 => a * sin(2π/3),
    vx2 => -a*ω*sin(2π/3),    vy2 => a*ω*cos(2π/3),
    
    # Body 3
    x3 => a * cos(4π/3),      y3 => a * sin(4π/3),
    vx3 => -a*ω*sin(4π/3),    vy3 => a*ω*cos(4π/3)
]

tspan = (0.0, 50.0)
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, ImplicitMidpoint(), dt=0.05)  

# ─── 3. Animate Chaotic Motion ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Lagrange Equilateral Triangle (Rotating Frame)",
    aspect=DataAspect(),
    limits=(-5..5, -5..5)
)

# Initial positions
pos1 = Observable(Point2f(sol[x1][1], sol[y1][1]))
pos2 = Observable(Point2f(sol[x2][1], sol[y2][1]))
pos3 = Observable(Point2f(sol[x3][1], sol[y3][1]))

# Trails with limited length
trail_length = 150
trail1 = Observable([Point2f(sol[x1][1], sol[y1][1])])
trail2 = Observable([Point2f(sol[x2][1], sol[y2][1])])
trail3 = Observable([Point2f(sol[x3][1], sol[y3][1])])

colors = [(:orange, :yellow), (:blue, :cyan), (:red, :pink)]

for (i, (pos, trail, (c1, c2))) in enumerate(zip([pos1, pos2, pos3], 
                                                [trail1, trail2, trail3], 
                                                colors))
    GLMakie.scatter!(ax, pos, color=c1, markersize=15, strokecolor=:black, strokewidth=1)
    GLMakie.lines!(ax, trail, color=c2, linewidth=1.5)
end

record(fig, "Lagrange Equilateral Triangle (Rotating Frame).gif", eachindex(sol.t); framerate=30) do i
    # Update positions
    pos1[] = Point2f(sol[x1][i], sol[y1][i])
    pos2[] = Point2f(sol[x2][i], sol[y2][i])
    pos3[] = Point2f(sol[x3][i], sol[y3][i])

    # Update trails (maintain limited length)
    for trail in [trail1, trail2, trail3]
        if length(trail[]) > trail_length
            trail[] = trail[][end-trail_length+1:end]
        end
    end
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])

    # Notify changes
    notify.([pos1, pos2, pos3])
    notify.([trail1, trail2, trail3])
end

Butterfly Choreography

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t

# ─── 1. Corrected System Definition ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Add ε softening to prevent NaN during close approaches
ε = 1e-6
dx12 = x2 - x1; dy12 = y2 - y1
dx13 = x3 - x1; dy13 = y3 - y1
dx23 = x3 - x2; dy23 = y3 - y2

r12³ = (dx12^2 + dy12^2 + ε^2)^(3/2)
r13³ = (dx13^2 + dy13^2 + ε^2)^(3/2)
r23³ = (dx23^2 + dy23^2 + ε^2)^(3/2)

eqs = [
    D(x1) ~ vx1, D(y1) ~ vy1,
    D(x2) ~ vx2, D(y2) ~ vy2,
    D(x3) ~ vx3, D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12³ + m3*dx13/r13³),
    D(vy1) ~ G*(m2*dy12/r12³ + m3*dy13/r13³),

    D(vx2) ~ G*(m3*dx23/r23³ - m1*dx12/r12³),
    D(vy2) ~ G*(m3*dy23/r23³ - m1*dy12/r12³),

    D(vx3) ~ G*(-m1*dx13/r13³ - m2*dx23/r23³),
    D(vy3) ~ G*(-m1*dy13/r13³ - m2*dy23/r23³),
]

@mtkbuild threebody = ODESystem(eqs, t)

# ─── 2. Precise Butterfly Parameters ───
Gval = 1.0
u0 = [
    x1 => 0.0,        y1 => 0.0,       
    vx1 => 0.513938,  vy1 => 0.304736,
    
    x2 => 0.470984,   y2 => -0.166094,
    vx2 => -0.358064, vy2 => 0.629851,
    
    x3 => -0.470984,  y3 => 0.166094,
    vx3 => -0.358064, vy3 => 0.629851
]

tspan = (0.0, 17.0656)  # Exact orbital period
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

# ─── 3. High-Precision Solving ───
prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, Vern9(), reltol=1e-12, abstol=1e-12, saveat=0.01)

# ─── 4. Optimized Visualization ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Butterfly Choreography Orbit",
    aspect=DataAspect(),
    limits=(-2.5..2.5, -2.5..2.5)  # Tightened view
)

current_frame = Observable(1)
pos1 = @lift Point2f(sol[x1][$current_frame], sol[y1][$current_frame])
pos2 = @lift Point2f(sol[x2][$current_frame], sol[y2][$current_frame])
pos3 = @lift Point2f(sol[x3][$current_frame], sol[y3][$current_frame])

trail_length = 200
trail1 = Observable(Point2f[])
trail2 = Observable(Point2f[])
trail3 = Observable(Point2f[])

colors = [:gold, :cyan, :magenta]

GLMakie.scatter!(ax, pos1, color=colors[1], markersize=15, label="Body 1")
GLMakie.scatter!(ax, pos2, color=colors[2], markersize=15, label="Body 2")
GLMakie.scatter!(ax, pos3, color=colors[3], markersize=15, label="Body 3")

GLMakie.lines!(ax, trail1, color=(colors[1], 0.4), linewidth=2)
GLMakie.lines!(ax, trail2, color=(colors[2], 0.4), linewidth=2)
GLMakie.lines!(ax, trail3, color=(colors[3], 0.4), linewidth=2)

axislegend(ax, position=:rt)

record(fig, "butterfly_choreography.gif", eachindex(sol.t); framerate=30) do i
    current_frame[] = i
    
    # Update trails
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])
    
    # Maintain trail length
    foreach(trail -> length(trail[]) > trail_length && popfirst!(trail[]), 
           [trail1, trail2, trail3])
    
    notify.([trail1, trail2, trail3])
end

Schubart's Periodic Orbit

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t

# ─── 1. System Definition with Softening ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Add ε softening to prevent singularity
ε = 1e-8
dx12 = x2 - x1; dy12 = y2 - y1
dx13 = x3 - x1; dy13 = y3 - y1
dx23 = x3 - x2; dy23 = y3 - y2

r12³ = (dx12^2 + dy12^2 + ε^2)^(3/2)
r13³ = (dx13^2 + dy13^2 + ε^2)^(3/2)
r23³ = (dx23^2 + dy23^2 + ε^2)^(3/2)

eqs = [
    D(x1) ~ vx1, D(y1) ~ vy1,
    D(x2) ~ vx2, D(y2) ~ vy2,
    D(x3) ~ vx3, D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12³ + m3*dx13/r13³),
    D(vy1) ~ G*(m2*dy12/r12³ + m3*dy13/r13³),

    D(vx2) ~ G*(m3*dx23/r23³ - m1*dx12/r12³),
    D(vy2) ~ G*(m3*dy23/r23³ - m1*dy12/r12³),

    D(vx3) ~ G*(-m1*dx13/r13³ - m2*dx23/r23³),
    D(vy3) ~ G*(-m1*dy13/r13³ - m2*dy23/r23³),
]

@mtkbuild threebody = ODESystem(eqs, t)

# ─── 2. Precision Parameters ───
Gval = 1.0
u0 = [
    x1 => -1.0, y1 => 0.0, vx1 => 0.0, vy1 => 0.347,
    x2 => 1.0,  y2 => 0.0, vx2 => 0.0, vy2 => -0.347,
    x3 => 0.0,  y3 => 0.0, vx3 => 0.0, vy3 => 0.694
]

tspan = (0.0, 6.3259*4)  # 4 orbital periods
p = [G => Gval, m1 => 1.0, m2 => 1.0, m3 => 1.0]

# ─── 3. High-Precision Solving ───
prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, Vern9(), reltol=1e-12, abstol=1e-12, saveat=0.01)

# ─── 4. Optimized Visualization ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Schubart's Periodic Orbit(1:1:1 Mass Ratio)",
    aspect=DataAspect(),
    limits=(-1.5..1.5, -1.5..1.5)  # Tight view
)

current_frame = Observable(1)
pos1 = @lift Point2f(sol[x1][$current_frame], sol[y1][$current_frame])
pos2 = @lift Point2f(sol[x2][$current_frame], sol[y2][$current_frame])
pos3 = @lift Point2f(sol[x3][$current_frame], sol[y3][$current_frame])

trail_length = 200
trail1 = Observable(Point2f[])
trail2 = Observable(Point2f[])
trail3 = Observable(Point2f[])

colors = [:gold, :cyan, :magenta]

GLMakie.scatter!(ax, pos1, color=colors[1], markersize=20, label="1M")
GLMakie.scatter!(ax, pos2, color=colors[2], markersize=15, label="1M")
GLMakie.scatter!(ax, pos3, color=colors[3], markersize=15, label="1M")

GLMakie.lines!(ax, trail1, color=(colors[1], 0.4), linewidth=2)
GLMakie.lines!(ax, trail2, color=(colors[2], 0.4), linewidth=2)
GLMakie.lines!(ax, trail3, color=(colors[3], 0.4), linewidth=2)

axislegend(ax, position=:rt)

record(fig, "Schubart's Periodic Orbit.gif", eachindex(sol.t); framerate=30) do i
    current_frame[] = i
    
    # Efficient trail updates
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])
    
    foreach(trail -> length(trail[]) > trail_length && popfirst!(trail[]), 
           [trail1, trail2, trail3])
    
    notify.([trail1, trail2, trail3])
end

Mothra Orbit (Lemniscate)

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: D_nounits as D, t_nounits as t

# ─── 1. System Definition with Softening ───
@parameters G m1 m2 m3 
@variables x1(t) y1(t) vx1(t) vy1(t)
@variables x2(t) y2(t) vx2(t) vy2(t)
@variables x3(t) y3(t) vx3(t) vy3(t)

# Add ε softening to prevent singularity
ε = 1e-8
dx12 = x2 - x1; dy12 = y2 - y1
dx13 = x3 - x1; dy13 = y3 - y1
dx23 = x3 - x2; dy23 = y3 - y2

r12³ = (dx12^2 + dy12^2 + ε^2)^(3/2)
r13³ = (dx13^2 + dy13^2 + ε^2)^(3/2)
r23³ = (dx23^2 + dy23^2 + ε^2)^(3/2)

eqs = [
    D(x1) ~ vx1, D(y1) ~ vy1,
    D(x2) ~ vx2, D(y2) ~ vy2,
    D(x3) ~ vx3, D(y3) ~ vy3,

    D(vx1) ~ G*(m2*dx12/r12³ + m3*dx13/r13³),
    D(vy1) ~ G*(m2*dy12/r12³ + m3*dy13/r13³),

    D(vx2) ~ G*(m3*dx23/r23³ - m1*dx12/r12³),
    D(vy2) ~ G*(m3*dy23/r23³ - m1*dy12/r12³),

    D(vx3) ~ G*(-m1*dx13/r13³ - m2*dx23/r23³),
    D(vy3) ~ G*(-m1*dy13/r13³ - m2*dy23/r23³),
]

@mtkbuild threebody = ODESystem(eqs, t)

# ─── 2. Precision Parameters ───
Gval = 1.0
u0 = [
    x1 => -0.989366, y1 => 0.249308, 
    vx1 => 0.457354, vy1 => 0.430348,
    x2 => 0.989366,  y2 => -0.249308,
    vx2 => 0.457354, vy2 => 0.430348,
    x3 => 0.0,       y3 => 0.0,
    vx3 => -0.914708, vy3 => -0.860696
]

tspan = (0.0, 6.3259*4)  # 4 orbital periods
p = [G => Gval, m1 => 2.0, m2 => 1.0, m3 => 1.0]

# ─── 3. High-Precision Solving ───
prob = ODEProblem(threebody, u0, tspan, p)
sol = solve(prob, Vern9(), reltol=1e-12, abstol=1e-12, saveat=0.01)

# ─── 4. Optimized Visualization ───
fig = Figure(size=(800, 800))
ax = Axis(fig[1,1], 
    title="Mothra Orbit (2:1:1 Mass Ratio)",
    aspect=DataAspect(),
    limits=(-5..5, -5..5)  # Tight view
)

current_frame = Observable(1)
pos1 = @lift Point2f(sol[x1][$current_frame], sol[y1][$current_frame])
pos2 = @lift Point2f(sol[x2][$current_frame], sol[y2][$current_frame])
pos3 = @lift Point2f(sol[x3][$current_frame], sol[y3][$current_frame])

trail_length = 200
trail1 = Observable(Point2f[])
trail2 = Observable(Point2f[])
trail3 = Observable(Point2f[])

colors = [:gold, :cyan, :magenta]

GLMakie.scatter!(ax, pos1, color=colors[1], markersize=20, label="2M")
GLMakie.scatter!(ax, pos2, color=colors[2], markersize=15, label="1M")
GLMakie.scatter!(ax, pos3, color=colors[3], markersize=15, label="1M")

GLMakie.lines!(ax, trail1, color=(colors[1], 0.4), linewidth=2)
GLMakie.lines!(ax, trail2, color=(colors[2], 0.4), linewidth=2)
GLMakie.lines!(ax, trail3, color=(colors[3], 0.4), linewidth=2)

axislegend(ax, position=:rt)

record(fig, "mothra_orbit.gif", eachindex(sol.t); framerate=30) do i
    current_frame[] = i
    
    # Efficient trail updates
    push!(trail1[], pos1[])
    push!(trail2[], pos2[])
    push!(trail3[], pos3[])
    
    foreach(trail -> length(trail[]) > trail_length && popfirst!(trail[]), 
           [trail1, trail2, trail3])
    
    notify.([trail1, trail2, trail3])
end